In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import statsmodels.api as sm
import sqlite3
import os
import re
import seaborn as sns
import matplotlib.pyplot as plt
from networkx.algorithms import bipartite
from itertools import combinations

import warnings
warnings.filterwarnings('ignore')

from utils import ROOTPATH, build_df_year, generate_index
from backbone import disparity_filter

## load data

In [2]:
dfall = pd.DataFrame()
for y in range(2012, 2016):
    dfyear = build_df_year(y, with_text=False, with_time=True)
    dfall = dfall.append(dfyear)

building base dataframe for year 2012...
- fetching submissions for year 2012
- fetching comments for year 2012
building base dataframe for year 2013...
- fetching submissions for year 2013
- fetching comments for year 2013
building base dataframe for year 2014...
- fetching submissions for year 2014
- fetching comments for year 2014
building base dataframe for year 2015...
- fetching submissions for year 2015
- fetching comments for year 2015


In [7]:
# select a subset of users
# aggr_func = {
#     "subreddit": lambda x: set(x),
#     "submission_id": lambda x: list(x),
#     "comment_id": lambda x: list(x)
#             }
# user_aggr = dfall.groupby("author").agg(aggr_func)  # this should take a while
# user_aggr = user_aggr.reset_index()
# user_aggr["n_subr"] = user_aggr["subreddit"].map(lambda x: len(x))
# user_aggr["n_uniq_subm"] = user_aggr["submission_id"].map(lambda x: len(set(x)))
# user_aggr["n_total_post"] = user_aggr["submission_id"].map(lambda x: len(x))

# user_select1 = user_aggr[(user_aggr["n_subr"]>=2)&(user_aggr["n_total_post"]>=10)][["author"]]
# user_select2 = user_aggr[(user_aggr["n_subr"]>=2)&(user_aggr["n_total_post"]>=20)][["author"]]
# user_select1.to_csv(ROOTPATH + "indx/ACTIVEUSERS_ALLYEARS10.csv", index=False)
# user_select2.to_csv(ROOTPATH + "indx/ACTIVEUSERS_ALLYEARS20.csv", index=False)

In [ ]:
# filter users
user_select = pd.read_csv(ROOTPATH + "indx/ACTIVEUSERS_ALLYEARS20.csv")
dfall = dfall[dfall["author"].isin(user_select)]

In [ ]:
dfall